In [7]:
import pandas as pd
from tqdm import tqdm
import numpy as np

# Weather

In [8]:
# Data from NYC - Central Park https://www.weather.gov/wrh/Climate?wfo=okx
df_weather = pd.read_csv('../data/external/weather_data_janfev2024.csv', delimiter=';')

In [9]:
df_weather.head()

,Date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
0,01/01/2024,45,32,38.5,5.4,26,0,0,0,0
1,02/01/2024,43,26,34.5,1.6,30,0,0,0,0
2,03/01/2024,45,28,36.5,3.8,28,0,0,0,0
3,04/01/2024,45,26,35.5,2.9,29,0,T,T,0
4,05/01/2024,38,21,29.5,-2.9,35,0,0,0,0


In [10]:
df_weather = df_weather.rename(columns={'New Snow' : 'new_snow',
                                        'Snow Depth' : 'snow_depth'})

In [11]:
df_weather.to_parquet('../data/external/weather_unify.parquet.gzip', compression='gzip', index=False)

# Streets Address

In [5]:
df = pd.read_csv('../data/external/taxi_zone_lookup.csv', dtype=str)

In [6]:
df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
